In [131]:
# Include Libraries
import numpy as np
import tensorflow as tf
import gym
import tensorflow.contrib.layers as layers
from mujoco_py import load_model_from_xml, MjSim, MjViewer
import policies
import value_functions as vfuncs
import utils_pg as utils

# Environment setup
#env = "CartPole-v0"
#env="InvertedPendulum-v2"
env = "Walker2d-v2"

# discrete = isinstance(env.action_space, gym.spaces.Discrete)
# observation_dim = env.observation_space.shape[0]
# action_dim = env.action_space.n if discrete else env.action_space.shape[0]
max_ep_len = 1000
num_traj = 10
#traj_length = max_ep_len*(observation_dim + 2)
latent_size = 64
use_baseline = True



In [132]:
# Feed forward network (multi-layer-perceptron, or mlp)

def build_mlp(mlp_input,output_size,scope,n_layers,size,output_activation=None):
    '''
    Build a feed forward network
    '''
    Input = mlp_input
    with tf.variable_scope(scope):
        # Dense Layers
        for i in range(n_layers-1):
            dense = tf.layers.dense(inputs = Input, units = size, activation = tf.nn.relu, bias_initializer=tf.constant_initializer(1.0))
            Input = dense
        # Fully Connected Layer
        out = layers.fully_connected(inputs = Input, num_outputs = output_size, activation_fn=output_activation)
    return out

In [133]:
class MetaLearner():
    def __init__(self, env, max_ep_len, num_traj,latent_size ):
        self.env = gym.make(env)
        self.discrete = isinstance(self.env.action_space, gym.spaces.Discrete)
        self.observation_dim = self.env.observation_space.shape[0]
        self.action_dim = self.env.action_space.n if self.discrete else self.env.action_space.shape[0]
        self.max_ep_len = max_ep_len
        self.num_traj = num_traj
        self.traj_length = self.max_ep_len*(self.observation_dim + 2) # TO Change
        self.use_baseline = True
        self.latent_size = latent_size
        self.feature_size = self.observation_dim + 1 + self.env.action_space.shape[0] # HC
        self.lr = 3e-3
        self.num_layers = 1
        self.layers_size = 32
        self.num_mdps = 10
        self.model = self.env.env.model
        self.sim = MjSim(self.model)
        self.gamma = 0.95
        self.decoder_len = 32

        # build model
        self.ConstructGraph()
    
    def add_placeholders(self):
        self.observation_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        if(self.discrete):
            self.action_placeholder_explore = tf.placeholder(tf.int32, shape=(None))
            self.action_placeholder_exploit = tf.placeholder(tf.int32, shape=(None))
        else:
            self.action_placeholder_explore = tf.placeholder(tf.float32, shape=(None,self.action_dim))
            self.action_placeholder_exploit= tf.placeholder(tf.float32, shape=(None,self.action_dim))

        self.baseline_target_placeholder = tf.placeholder(tf.float32, shape= None)
        self.advantage_placeholder_explore = tf.placeholder(tf.float32, shape=(None))
        
        #self.encoder_input_placeholder = tf.placeholder(tf.float32, shape= (self.num_traj,self.traj_length))
        self.encoder_input_placeholder = tf.placeholder(tf.float32, [None, None, self.feature_size])
        self.decoder_input_placeholder = tf.placeholder(tf.float32, shape= (1,self.latent_size))
        self.sequence_length_placeholder = tf.placeholder(tf.int32, [None, ])
        
        self.observation_placeholder_exploit = tf.placeholder(tf.float32, shape=(None,self.observation_dim))
        #TODO
        self.advantage_placeholder_exploit = tf.placeholder(tf.float32, shape=(None))
        
        
    def build_policy_explore(self, scope = "policy_explore"):
        if (self.discrete):
            self.action_logits = build_mlp(self.observation_placeholder_explore,self.action_dim,scope = scope,n_layers=self.num_layers,size = self.layers_size,output_activation=None)
            self.explore_action = tf.multinomial(self.action_logits,1)
            self.explore_action = tf.squeeze(self.explore_action, axis=1)
            self.explore_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.action_logits, labels = self.action_placeholder_explore)

        else:   
            action_means = build_mlp(self.observation_placeholder_explore,self.action_dim,scope,n_layers=self.num_layers, size = self.layers_size,output_activation=None)
            init = tf.constant(np.random.rand(1, 2))
            #log_std = tf.get_variable("log_std", [self.action_dim])
            log_std = tf.get_variable(
                'log_std',
                shape=[1, self.action_dim],
                dtype=tf.float32,
                initializer=tf.zeros_initializer(),
                trainable=True
            )
            log_std = tf.log(tf.exp(log_std) + 1.)
            log_std = tf.tile(log_std,
                                   [tf.shape(action_means)[0], 1])
            self.explore_action = tf.random_normal((1,),
                                                   action_means,
                                                   log_std)
            dist = tf.contrib.distributions.MultivariateNormalDiag(action_means, log_std)
            self.explore_logprob = dist.prob(self.action_placeholder_explore, name = 'log_prob')
            #self.explore_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            #mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
            #self.explore_logprob =  mvn.log_prob(value = self.action_placeholder_explore, name='log_prob')

    
    
    def build_policy_exploit(self, scope = "policy_exploit"):
        if(self.discrete):
            #self.exploit_action_logits = (tf.matmul(tf.nn.relu(tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W1) + self.d_B1), self.d_W2) + self.d_B2),self.d_W3) + self.d_B3)
            #self.exploit_action_logits = tf.matmul(self.observation_placeholder_exploit,self.d_W3) + self.d_B3
            self.exploit_action_logits = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            self.exploit_action = tf.multinomial(self.exploit_action_logits,1)
            self.exploit_action = tf.squeeze(self.exploit_action, axis=1)
            self.exploit_logprob = -tf.nn.sparse_softmax_cross_entropy_with_logits(logits = self.exploit_action_logits, labels = self.action_placeholder_exploit)
        else:
            action_means = tf.matmul(tf.nn.relu(tf.matmul(self.observation_placeholder_exploit,self.d_W2) + self.d_B2), self.d_W3) + self.d_B3
            init = tf.constant(np.random.rand(1, 2))
            log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
            self.exploit_action = action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
            self.exploit_logprob = utils.gauss_log_prob(mu=action_means, logstd=log_std, x=self.action_placeholder_exploit)
            
#             init = tf.constant(np.random.rand(1, 2))
#             log_std = tf.get_variable("exploit_log_prob", [self.action_dim])
#             self.exploit_action =   action_means + tf.multiply(tf.exp(log_std),tf.random_normal(shape = (self.action_dim,1),mean=0,stddev=1))
#             mvn = tf.contrib.distributions.MultivariateNormalDiag(action_means, tf.exp(log_std))
#             self.exploit_logprob =  mvn.log_prob(value = self.action_placeholder_exploit, name='exploit_log_prob')

        #self.loss_grads_exploit = self.exploit_logprob * self.advantage_placeholder_exploit
        
    def NNEncoder(self, scope = "NNEncoder"):
        self.Z = build_mlp(self.encoder_input_placeholder,self.latent_size,scope = scope,n_layers=3,size = 60,output_activation=None)
    
 

    # input [num_traj, length, features (obs + action + reward) ]
    def LSTMEncoder(self, scope = "LSTMEncoder"):
        self.hidden_size = 64
        initializer = tf.random_uniform_initializer(-1, 1)
        cell = tf.nn.rnn_cell.LSTMCell(self.hidden_size, self.feature_size, initializer=initializer)
        cell_out = tf.contrib.rnn.OutputProjectionWrapper(cell, self.latent_size)
        self.output, _ = tf.nn.dynamic_rnn(cell_out,self.encoder_input_placeholder,self.sequence_length_placeholder,dtype=tf.float32,)
        batch_size = tf.shape(self.output)[0]
        max_length = tf.shape(self.output)[1]
        out_size = int(self.output.get_shape()[2])
        index = tf.range(0, batch_size) * max_length + (self.sequence_length_placeholder - 1)
        flat = tf.reshape(self.output, [-1, out_size])
        self.Z = tf.reduce_mean(tf.gather(flat, index), axis = 0)
        
    
    def Decoder(self, decoder_out_dim, scope):
        return build_mlp(self.decoder_input_placeholder,decoder_out_dim,scope = scope,n_layers=1,size = 64,output_activation=None)
    
    
    def sample_paths_explore(self, env,mdp_num,Test = False, num_episodes = None):
        paths = []
        self.length = []
        episode_rewards = []
        #self.sim.model.opt.gravity[-1] = -5 - mdp_num
        for i in range(self.num_traj):
            pad = False
            state = env.reset()
            new_states,states,new_actions, actions, rewards,new_rewards = [], [], [], [], [],[]
            episode_reward = 0
            for step in range(self.max_ep_len):
                if (pad):
                    states.append([0]*self.observation_dim)
                    if(self.discrete):
                        actions.append(0)
                    else:
                        actions.append([0]*self.action_dim)
                    rewards.append(0)
                else:
                    states.append(state)
                    new_states.append(state)
                    #action = self.sess.run(self.explore_action, feed_dict={self.observation_placeholder_explore : states[-1][None]})[0]
                    action = self.policyfn.sample_action(state) #NEW
                    state, reward, done, info = env.step(action)
                    actions.append(action)
                    new_actions.append(action)
                    rewards.append(reward)
                    new_rewards.append(reward)
                    episode_reward += reward
                    if (done or step == self.max_ep_len-1):
                        episode_rewards.append(episode_reward)
                        self.length.append(step + 1)
                        pad = True 
                        
            episode_rewards.append(episode_reward)            
            #print("explore",np.array(actions))
            path = {"new_obs" : np.array(new_states),"observation" : np.array(states),
                                "reward" : np.array(rewards),"new_acs" : np.array(new_actions),
                                "action" : np.array(actions),"new_rewards":np.array(new_rewards)}
            paths.append(path)
        return paths, episode_rewards
    
    def sample_paths_exploit(self, env,Z,mdp_num,Test = False, num_episodes = None):
        #self.sim.model.opt.gravity[-1] = -5 - mdp_num
        
        paths = []
        num = 0
        episode_rewards = []
        for i in range(self.num_traj):
            state = env.reset()
            states, actions, rewards = [], [], []
            episode_reward = 0
            for step in range(self.max_ep_len):
                states.append(state)
                action = self.sess.run(self.exploit_action, feed_dict={self.observation_placeholder_exploit : state[None], self.decoder_input_placeholder: Z})[0]
                state, reward, done, info = env.step(action)
                actions.append(action)
                rewards.append(reward)
                episode_reward += reward
                if (done or step == self.max_ep_len-1):
                    episode_rewards.append(episode_reward)
                    break
            #print("exploit",np.array(actions))
            path = {"observation" : np.array(states),
                                "reward" : np.array(rewards),
                                "action" : np.array(actions)}
            paths.append(path)
 
        #print("exploit success: ", num)
        return paths, episode_rewards
    
    def get_returns(self,paths, explore = False):
        all_returns = []
        m = 0
        for path in paths:
            rewards = path["reward"]
            returns = []
            if(explore):
                length = self.length[m]
            else:
                length = len(rewards)
            for i in range(length):
                path_returns = 0
                k = 0
                for j in range(i,length):
                    path_returns = path_returns + rewards[j]*(self.gamma)**k
                    k = k+1
                returns.append(path_returns)
            all_returns.append(returns)
            m+=1
        returns = np.concatenate(all_returns)
        return returns
    
    def stack_trajectories(self,paths):
        trajectories = []
        for path in paths:
            rewards = path["reward"]
            states = path["observation"]
            action = path["action"]
            
            SAR = []
            for i in range(len(states)):
                SAR.append(list(states[i]) + list(action[i]) + [rewards[i]])
            trajectories.append(SAR)
        return np.array(trajectories)
    
    def addBaseline(self):
        self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=self.num_layers, size = self.layers_size,output_activation=None)
        self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
        baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
        self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)

    def calculate_advantage(self,returns, observations):
        if (self.use_baseline):
            baseline = self.sess.run(self.baseline, {input_placeholder:observations})
            adv = returns - baseline
            adv = (adv - np.mean(adv))/np.std(adv)
        else:
            adv = returns
        return adv
    

        
    def ConstructGraph(self):
        tf.reset_default_graph()
        self.sess = tf.Session()
        
        self.vf = vfuncs.NnValueFunction(session=self.sess,ob_dim=self.observation_dim)
        self.policyfn = policies.GaussianPolicy(self.sess,self.observation_dim, self.action_dim)
        
        self.add_placeholders()
        
#         self.add_placeholders()
        
#         self.build_policy_explore()
#         self.explore_policy_loss = -tf.reduce_sum(self.explore_logprob * self.advantage_placeholder_explore)
#         self.loss_grads_explore = -self.explore_logprob * self.advantage_placeholder_explore
#         self.tvars_explore = tf.trainable_variables()
#         self.gradients_explore = tf.gradients(self.explore_policy_loss,self.tvars_explore)
        
#         #self.addBaseline()
        
#         self.baseline = build_mlp(self.observation_placeholder_explore,1,scope = "baseline",n_layers=1, size = 16,output_activation=None)
#         self.baseline_loss = tf.losses.mean_squared_error(self.baseline_target_placeholder,self.baseline,scope = "baseline")
#         baseline_adam_optimizer =  tf.train.AdamOptimizer(learning_rate = self.lr)
#         self.update_baseline_op = baseline_adam_optimizer.minimize(self.baseline_loss)
        
#         #Encoder LSTM
        self.LSTMEncoder()
        
        
        #decoder weights
        #self.d_W1 = self.Decoder(scope = "W1", decoder_out_dim = self.observation_dim*self.decoder_len)
        self.d_W2 = self.Decoder(scope = "W2", decoder_out_dim = self.observation_dim*self.decoder_len)
        #self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*action_dim)
        self.d_W3 = self.Decoder(scope = "W3", decoder_out_dim = self.decoder_len*self.action_dim)
        
        #self.d_W1 = ((self.d_W1 - (tf.reduce_max(self.d_W1) + tf.reduce_min(self.d_W1))/2)/(tf.reduce_max(self.d_W1) - tf.reduce_min(self.d_W1)))*2 
        #self.d_W2 = ((self.d_W2 - (tf.reduce_max(self.d_W2) + tf.reduce_min(self.d_W2))/2)/(tf.reduce_max(self.d_W2) - tf.reduce_min(self.d_W2)))*2 
        #self.d_W3 = ((self.d_W3 - (tf.reduce_max(self.d_W3) + tf.reduce_min(self.d_W3))/2)/(tf.reduce_max(self.d_W3) - tf.reduce_min(self.d_W3)))*2 
        
        #self.d_W1 = tf.reshape(self.d_W1, [self.observation_dim, self.decoder_len])
        self.d_W2 = tf.reshape(self.d_W2, [self.observation_dim, self.decoder_len])
        #self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        self.d_W3 = tf.reshape(self.d_W3, [self.decoder_len, self.action_dim])
        
        # decoder output bias
        #self.d_B1 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B1"), [self.decoder_len])
        self.d_B2 = tf.reshape(self.Decoder(decoder_out_dim = self.decoder_len, scope = "B2"), [self.decoder_len])
        self.d_B3 = tf.reshape(self.Decoder(decoder_out_dim = self.action_dim, scope = "B3"), [self.action_dim])
        #self.d_B1 = ((self.d_B1 - (tf.reduce_max(self.d_B1) + tf.reduce_min(self.d_B1))/2)/(tf.reduce_max(self.d_B1) - tf.reduce_min(self.d_B1)))*2 
        #self.d_B2 = ((self.d_B2 - (tf.reduce_max(self.d_B2) + tf.reduce_min(self.d_B2))/2)/(tf.reduce_max(self.d_B2) - tf.reduce_min(self.d_B2)))*2 
        
        
        # exploit policy
        self.build_policy_exploit()
        #self.d = [self.d_W1, self.d_B1, self.d_W2, self.d_B2, self.d_W3, self.d_B3]
        self.exploit_policy_loss = -tf.reduce_sum(self.exploit_logprob * self.advantage_placeholder_exploit)
        self.d = [self.d_W3, self.d_B3]
        self.gradients_exploit = tf.gradients(self.exploit_policy_loss,self.d)
        
        # train encoder and decoder
        adam_optimizer_exploit =  tf.train.AdamOptimizer(self.lr*0.05)
        self.output_train_op = adam_optimizer_exploit.minimize(self.exploit_policy_loss)
        # train original network
#         adam_optimizer_explore = tf.train.AdamOptimizer(self.lr)
#         self.input_train_op = adam_optimizer_explore.minimize(self.explore_policy_loss)
        init = tf.global_variables_initializer()
        self.sess.run(init)
    
    def initialize(self):
        self.ConstructGraph()
#         # create tf session
#         self.sess = tf.Session()
        
#         # initiliaze all variables
#         init = tf.global_variables_initializer()
#         self.sess.run(init)
    
    def train_step(self): 
        # sample num_traj*num_MDPs
        for i in range(self.num_mdps):
            explore_paths, explore_rewards = self.sample_paths_explore(self.env,i)
            observations_explore = np.concatenate([path["observation"] for path in explore_paths])
            new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
            new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
            actions_explore = np.concatenate([path["action"] for path in explore_paths])
            rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
            
            vtargs, vpreds, advantages_explore = [], [], []
            for path in explore_paths:
                rew_t = path["new_rewards"]
                return_t = utils.discount(rew_t, self.gamma)
                vpred_t = self.vf.predict(path["new_obs"])
                adv_t = return_t - vpred_t
                advantages_explore.append(adv_t)
                vtargs.append(return_t)
                vpreds.append(vpred_t)
                
                
            #returns_explore = self.get_returns(explore_paths, explore = True)
            advantages_explore = np.concatenate(advantages_explore)
            advantages_explore = (advantages_explore - advantages_explore.mean()) / (advantages_explore.std() + 1e-8)
            vtarg_n = np.concatenate(vtargs)
            vpred_n = np.concatenate(vpreds)
            #update baseline
            #print(advantages_explore.shape,new_observations_explore.shape)
            self.vf.fit(new_observations_explore, vtarg_n)
            
            surr_loss, oldmean_na, oldlogstd_a = self.policyfn.update_policy(
                    new_observations_explore, new_actions_explore, advantages_explore, self.lr)
            
            print("average reward explore:", np.mean(explore_rewards))
            
            #print(returns_explore)
#             print("average reward explore: MDP", i, np.sum(explore_rewards)/num_traj, len(explore_rewards))

#             baseline_explore = self.sess.run(self.baseline, {self.observation_placeholder_explore:new_observations_explore})
#             adv = returns_explore - np.squeeze(baseline_explore)
#             advantages_explore = (adv - np.mean(adv))/(np.std(adv) + 1e-12)


#             # update the baseline

#             self.sess.run(self.update_baseline_op, {self.observation_placeholder_explore:new_observations_explore, 
#                                            self.baseline_target_placeholder : returns_explore})

            # calculate explore gradients
    #         grads_explore = self.sess.run(self.gradients_explore, feed_dict={
    #                     self.observation_placeholder_explore : observations_explore,
    #                     self.action_placeholder_explore : actions_explore,
    #                     self.advantage_placeholder_explore : returns_explore})
            #print("explore",grads_explore )
            # form trajectory matrix

            M = np.array(self.stack_trajectories(explore_paths))


            #encoder LSTM
            Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
            Z = np.reshape(Z,[1,len(Z)])
#             #print("Z",Z)
#             #print(self.sess.run(self.d, feed_dict = {self.decoder_input_placeholder: Z}))
#             #print("d",self.d)
#             # sample paths
#             tvars = tf.trainable_variables()
#             tvars_vals = self.sess.run(tvars[-5:-1])
#             #print(tvars_vals)
            for j in range(10):
                exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,i)
                # get observations, actions and rewards
                observations_exploit = np.concatenate([path["observation"] for path in exploit_paths])
                actions_exploit = np.concatenate([path["action"] for path in exploit_paths])
                rewards_exploit = np.concatenate([path["reward"] for path in exploit_paths])
                returns_exploit = self.get_returns(exploit_paths)
                print("average reward exploit: MDP", i, np.mean(exploit_rewards), len(exploit_rewards))


#             # exploit grads
#     #         grads_exploit = self.sess.run(self.gradients_exploit,feed_dict={
#     #                     self.observation_placeholder_exploit : observations_exploit,
#     #                     self.action_placeholder_exploit : actions_exploit,
#     #                     self.advantage_placeholder_exploit : returns_exploit,
#     #                     self.decoder_input_placeholder: Z})

            #train encoder and decoder network
                self.sess.run(self.output_train_op, feed_dict={
                                self.observation_placeholder_exploit : observations_exploit,
                                self.action_placeholder_exploit : actions_exploit,
                                self.advantage_placeholder_exploit : returns_exploit,
                                self.decoder_input_placeholder: Z})

            # find advantage for input network
    #         advantage_explore = 0
    #         for i in range(len(grads_exploit)):
    #             l1 = grads_exploit[i]
    #             l2 = grads_explore[i]
    #             advantage_explore = advantage_explore + np.matmul(l1.flatten(), l2.flatten())

            # train input policy
#             self.sess.run(self.input_train_op, feed_dict={
#                             self.observation_placeholder_explore : new_observations_explore,
#                             self.action_placeholder_explore : new_actions_explore,
#                             self.advantage_placeholder_explore : advantages_explore})
    
    def test(self):
        explore_paths, explore_rewards = self.sample_paths_explore(self.env,20*np.random.rand())
        print("gravity: ",self.sim.model.opt.gravity )
        M = self.stack_trajectories(explore_paths)
        Z = self.sess.run(self.Z, feed_dict = {self.encoder_input_placeholder: M,self.sequence_length_placeholder: self.length })
        Z = np.reshape(Z,[1,len(Z)])
        # sample paths
        exploit_paths, exploit_rewards = self.sample_paths_exploit(self.env,Z,20*np.random.rand())
        print(exploit_rewards)
        print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
        
    def train(self):
        self.initialize()
        num_epochs = 200
        for epoch in range(num_epochs):
            self.train_step()
            

In [134]:
a = MetaLearner(env, max_ep_len, num_traj, latent_size)

WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.
WARN: gym.spaces.Box autodetected dtype as <class 'numpy.float32'>. Please provide explicit dtype.


In [135]:
a.train()

average reward explore: -1.1320883480610526
average reward exploit: MDP 0 -14.274819753628254 10
average reward exploit: MDP 0 -10.553365483602764 10
average reward exploit: MDP 0 -11.346338262858957 10
average reward exploit: MDP 0 -8.224616854545815 10
average reward exploit: MDP 0 -9.017437227343319 10
average reward exploit: MDP 0 -4.743108064870849 10
average reward exploit: MDP 0 1.7078127597622044 10
average reward exploit: MDP 0 -1.5675674377568076 10
average reward exploit: MDP 0 -0.04365109831180627 10
average reward exploit: MDP 0 27.830806276660887 10
average reward explore: 4.139010475981371
average reward exploit: MDP 1 -3.6213362374930327 10
average reward exploit: MDP 1 -2.4307151615649247 10
average reward exploit: MDP 1 13.441927546832195 10
average reward exploit: MDP 1 19.830845333912784 10
average reward exploit: MDP 1 7.475000930805192 10
average reward exploit: MDP 1 14.063242649530755 10
average reward exploit: MDP 1 20.371221182048668 10
average reward exploit:

average reward exploit: MDP 4 82.00699509636527 10
average reward exploit: MDP 4 42.56875557085026 10
average reward exploit: MDP 4 28.438812643865326 10
average reward exploit: MDP 4 2.0509939393040155 10
average reward explore: -34.143440759462095
average reward exploit: MDP 5 147.40140127786077 10
average reward exploit: MDP 5 113.4154967574539 10
average reward exploit: MDP 5 159.03550011949247 10
average reward exploit: MDP 5 105.96277305480149 10
average reward exploit: MDP 5 152.7440850781664 10
average reward exploit: MDP 5 104.89180728359501 10
average reward exploit: MDP 5 127.34296143871313 10
average reward exploit: MDP 5 114.29431869690393 10
average reward exploit: MDP 5 88.20632834963514 10
average reward exploit: MDP 5 133.02742581966686 10
average reward explore: 13.996243520075314
average reward exploit: MDP 6 175.101716381618 10
average reward exploit: MDP 6 110.19954596677584 10
average reward exploit: MDP 6 115.99902880163809 10
average reward exploit: MDP 6 123.24

average reward exploit: MDP 9 288.36964686191084 10
average reward exploit: MDP 9 175.36694652123043 10
average reward exploit: MDP 9 161.7153435475266 10
average reward exploit: MDP 9 291.0762214706276 10
average reward exploit: MDP 9 256.27718753357055 10
average reward exploit: MDP 9 258.8243773464753 10
average reward exploit: MDP 9 169.2719555051965 10
average reward explore: -13.7426299894252
average reward exploit: MDP 0 179.4042903433643 10
average reward exploit: MDP 0 154.89259078961877 10
average reward exploit: MDP 0 188.52241196951906 10
average reward exploit: MDP 0 166.9568846948272 10
average reward exploit: MDP 0 175.82130005001198 10
average reward exploit: MDP 0 195.66053809873068 10
average reward exploit: MDP 0 94.08344602086149 10
average reward exploit: MDP 0 128.41327266964836 10
average reward exploit: MDP 0 127.6273164755199 10
average reward exploit: MDP 0 115.46715390609397 10
average reward explore: -3.08339826499814
average reward exploit: MDP 1 113.543839

average reward exploit: MDP 4 196.22240835954457 10
average reward exploit: MDP 4 88.63342911490504 10
average reward exploit: MDP 4 160.4660419396227 10
average reward exploit: MDP 4 60.76773474649009 10
average reward exploit: MDP 4 70.78729849094175 10
average reward exploit: MDP 4 109.811350668225 10
average reward exploit: MDP 4 52.93318072423293 10
average reward exploit: MDP 4 85.91496036302182 10
average reward exploit: MDP 4 124.17965611745899 10
average reward exploit: MDP 4 203.06388061998786 10
average reward explore: 36.92407939259631
average reward exploit: MDP 5 177.88189256986806 10
average reward exploit: MDP 5 306.55589309964444 10
average reward exploit: MDP 5 245.9281410600507 10
average reward exploit: MDP 5 146.52593808604712 10
average reward exploit: MDP 5 288.23848326007067 10
average reward exploit: MDP 5 282.2335706074474 10
average reward exploit: MDP 5 225.76581961992466 10
average reward exploit: MDP 5 225.1241643603202 10
average reward exploit: MDP 5 247

average reward exploit: MDP 8 201.56511549826752 10
average reward exploit: MDP 8 187.5370850486926 10
average reward explore: 48.71943351980506
average reward exploit: MDP 9 248.98626916401577 10
average reward exploit: MDP 9 253.38956416701257 10
average reward exploit: MDP 9 256.93052197221175 10
average reward exploit: MDP 9 296.9734607596448 10
average reward exploit: MDP 9 240.8743362856033 10
average reward exploit: MDP 9 227.28287502959915 10
average reward exploit: MDP 9 270.63708844168383 10
average reward exploit: MDP 9 233.16394207324396 10
average reward exploit: MDP 9 282.5251183718673 10
average reward exploit: MDP 9 237.24381377211066 10
average reward explore: 23.834448658346993
average reward exploit: MDP 0 289.63239398913413 10
average reward exploit: MDP 0 212.65995508141182 10
average reward exploit: MDP 0 286.131303339244 10
average reward exploit: MDP 0 307.26328049067064 10
average reward exploit: MDP 0 263.42521927742257 10
average reward exploit: MDP 0 222.405

...

average reward exploit: MDP 9 308.8974358207264 10
average reward exploit: MDP 9 282.68983596675156 10
average reward explore: 394.01372002019326
average reward exploit: MDP 0 313.4649951687296 10
average reward exploit: MDP 0 303.70003597654454 10
average reward exploit: MDP 0 315.5852662838438 10
average reward exploit: MDP 0 299.92037289012745 10
average reward exploit: MDP 0 273.51152100025763 10
average reward exploit: MDP 0 310.753718129234 10
average reward exploit: MDP 0 263.6310392372264 10
average reward exploit: MDP 0 235.29613822711326 10
average reward exploit: MDP 0 236.37437531344193 10
average reward exploit: MDP 0 221.804237310833 10
average reward explore: 390.4624830661151
average reward exploit: MDP 1 240.71085715756894 10
average reward exploit: MDP 1 234.7629971106009 10
average reward exploit: MDP 1 184.8013442551175 10
average reward exploit: MDP 1 236.79790423092723 10
average reward exploit: MDP 1 204.77116365248594 10
average reward exploit: MDP 1 195.4472183

average reward exploit: MDP 4 200.15090112437127 10
average reward exploit: MDP 4 156.29415734449347 10
average reward exploit: MDP 4 228.08160801311692 10
average reward exploit: MDP 4 170.09524153468044 10
average reward exploit: MDP 4 205.0122182126075 10
average reward explore: 322.28725894978487
average reward exploit: MDP 5 175.74174829387704 10
average reward exploit: MDP 5 244.54397606117942 10
average reward exploit: MDP 5 182.8450060333756 10
average reward exploit: MDP 5 153.14170673342062 10
average reward exploit: MDP 5 207.910406596526 10
average reward exploit: MDP 5 127.16461179507981 10
average reward exploit: MDP 5 219.02621658165975 10
average reward exploit: MDP 5 246.03026068194262 10
average reward exploit: MDP 5 170.47372910026715 10
average reward exploit: MDP 5 109.17748369232629 10
average reward explore: 335.20187302966076
average reward exploit: MDP 6 216.08789557956487 10
average reward exploit: MDP 6 206.23060764225306 10
average reward exploit: MDP 6 228.

average reward exploit: MDP 9 153.6242580103804 10
average reward exploit: MDP 9 240.55439287182827 10
average reward exploit: MDP 9 195.13062131211535 10
average reward exploit: MDP 9 266.6392324228933 10
average reward exploit: MDP 9 277.01711858740794 10
average reward exploit: MDP 9 256.35960951304804 10
average reward exploit: MDP 9 291.1952088346031 10
average reward exploit: MDP 9 284.06903056987585 10
average reward explore: 356.6099636275333
average reward exploit: MDP 0 213.2977499402453 10
average reward exploit: MDP 0 229.87616948157455 10
average reward exploit: MDP 0 246.5770052792067 10
average reward exploit: MDP 0 233.9142821264848 10
average reward exploit: MDP 0 247.55686192240654 10
average reward exploit: MDP 0 222.37462446835775 10
average reward exploit: MDP 0 245.0160325019744 10
average reward exploit: MDP 0 251.04437496048257 10
average reward exploit: MDP 0 232.0556091286559 10
average reward exploit: MDP 0 217.02145634020985 10
average reward explore: 393.35

average reward explore: 337.2592155480123
average reward exploit: MDP 4 260.0847143459662 10
average reward exploit: MDP 4 246.44845240514223 10
average reward exploit: MDP 4 237.2597553592453 10
average reward exploit: MDP 4 241.18521562294455 10
average reward exploit: MDP 4 269.14327123340234 10
average reward exploit: MDP 4 284.8174798856461 10
average reward exploit: MDP 4 233.24185479527395 10
average reward exploit: MDP 4 253.32018821182368 10
average reward exploit: MDP 4 278.38750376538536 10
average reward exploit: MDP 4 273.0648106508973 10
average reward explore: 339.77793171033835
average reward exploit: MDP 5 156.78819648900148 10
average reward exploit: MDP 5 207.48431502027037 10
average reward exploit: MDP 5 152.4108249729101 10
average reward exploit: MDP 5 234.5826191574326 10
average reward exploit: MDP 5 264.6973702789875 10
average reward exploit: MDP 5 304.0158343761658 10
average reward exploit: MDP 5 194.69314701973875 10
average reward exploit: MDP 5 226.96510

average reward exploit: MDP 8 281.87529624400923 10
average reward exploit: MDP 8 175.13180823800528 10
average reward exploit: MDP 8 190.6755676573381 10
average reward explore: 386.65280290154084
average reward exploit: MDP 9 289.5162800143727 10
average reward exploit: MDP 9 254.25637419683645 10
average reward exploit: MDP 9 258.44056761666667 10
average reward exploit: MDP 9 290.0711635591882 10
average reward exploit: MDP 9 269.9752269914449 10
average reward exploit: MDP 9 267.44777779474487 10
average reward exploit: MDP 9 281.36677520903237 10
average reward exploit: MDP 9 267.0488853999072 10
average reward exploit: MDP 9 273.31565245899407 10
average reward exploit: MDP 9 298.0881142098388 10
average reward explore: 365.18588171806266
average reward exploit: MDP 0 234.77597753543495 10
average reward exploit: MDP 0 227.78969730090267 10
average reward exploit: MDP 0 259.35215677306945 10
average reward exploit: MDP 0 234.23658390854433 10
average reward exploit: MDP 0 257.56

AttributeError: 'MetaLearner' object has no attribute 'exploit_rewards'

In [92]:
for epoch in range(200):
        print("epoch number: ", epoch)
        a.train_step()

epoch number:  0
average reward explore: 197.4367190503391
average reward exploit: MDP 0 182.6423977032496 200
average reward exploit: MDP 0 177.83136573437966 200
average reward exploit: MDP 0 186.4590380054437 200
average reward explore: 208.62477861875507
average reward exploit: MDP 1 200.1499529335926 200
average reward exploit: MDP 1 203.96533608881452 200
average reward exploit: MDP 1 203.2714773634698 200
average reward explore: 206.44248577942912
average reward exploit: MDP 2 173.34431477504702 200
average reward exploit: MDP 2 180.10862083826888 200
average reward exploit: MDP 2 175.50692178789933 200
average reward explore: 214.38352005067472


KeyboardInterrupt: 

In [58]:
explore_paths, explore_rewards = a.sample_paths_explore(a.env,1)

In [59]:
print("average reward explore:", np.mean(explore_rewards))
M = a.stack_trajectories(explore_paths)

average reward explore: 46.3447291282569


In [60]:
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })

In [61]:
Z = np.reshape(Z,[1,len(Z)])

In [62]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)

In [63]:
print("average reward exploit", np.mean(exploit_rewards), len(exploit_rewards))

average reward exploit 152.95256858651558 800


In [91]:
a.num_traj = 200

In [40]:
for i in range(20):
    explore_paths, explore_rewards = a.sample_paths_explore(a.env,30*np.random.rand())
    print("gravity:", a.sim.model.opt.gravity)
    print("average reward explore:", np.mean(explore_rewards))
    M = a.stack_trajectories(explore_paths)
    Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
    Z = np.reshape(Z,[1,len(Z)])
    exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1)
    print("average reward exploit", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))
    

gravity: [  0.           0.         -11.78211953]
average reward explore: 1304.1956304786172
average reward exploit 8011.892223745158 800
gravity: [  0.           0.         -19.76739288]
average reward explore: 513.5210888464073
average reward exploit 8236.849973173716 800
gravity: [ 0.          0.         -5.67745507]
average reward explore: 1733.9094007923966
average reward exploit 9397.641993205756 800
gravity: [  0.         0.       -33.888614]
average reward explore: 290.24741591480824
average reward exploit 9128.682414141625 800
gravity: [  0.           0.         -14.66682571]
average reward explore: 892.629266461097
average reward exploit 9177.183052697012 800
gravity: [  0.           0.         -16.68177216]
average reward explore: 690.623843804044
average reward exploit 9046.119152302825 800
gravity: [  0.           0.         -11.42352458]
average reward explore: 1286.0432377482339
average reward exploit 6973.652444899903 800
gravity: [  0.           0.         -34.55160267

KeyboardInterrupt: 

In [ ]:
observations_explore = np.concatenate([path["observation"] for path in explore_paths])
new_observations_explore = np.concatenate([path["new_obs"] for path in explore_paths])
new_actions_explore = np.concatenate([path["new_acs"] for path in explore_paths])
actions_explore = np.concatenate([path["action"] for path in explore_paths])
rewards_explore = np.concatenate([path["reward"] for path in explore_paths])
returns_explore = a.get_returns(explore_paths, explore = True)

In [ ]:
M = np.array(a.stack_trajectories(explore_paths))
Z = a.sess.run(a.Z, feed_dict = {a.encoder_input_placeholder: M,a.sequence_length_placeholder: a.length })
Z = np.reshape(Z,[1,len(Z)])

In [ ]:
 print("average reward explore: MDP", np.sum(explore_rewards)/num_traj, len(explore_rewards))

In [ ]:
exploit_paths, exploit_rewards = a.sample_paths_exploit(a.env,Z,1000)

In [ ]:
print("average reward exploit: MDP", np.sum(exploit_rewards) / num_traj, len(exploit_rewards))

In [ ]:
def randomize_pendulum(env):
    m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])
    l = config['pendulum_len_min'] + np.random.rand()*(config['pendulum_len_max'] - config['pendulum_len_min'])
    env.m = m
    env.l = l

In [ ]:
def randomize_hopper(env):
    ts = config['torso_min'] + np.random.rand()*(config['torso_max'] - config['torso_min'])
    f = config['friction_min'] + np.random.rand()*(config['friction_max'] - config['friction_min'])
    
    env.friction = f
    env.torso_size = ts
    env.apply_env_modifications()

In [ ]:
import yaml
cfg_filename = 'hopper-config.yml'
with open(cfg_filename,'r') as ymlfile:
    config = yaml.load(ymlfile)

In [ ]:
env = Randomizer(gym.make('Hopper-v2'), randomize_hopper)
env.unwrapped.__dict__.keys()
env.reset()

In [ ]:
m = config['pendulum_mass_min'] + np.random.rand()*(config['pendulum_mass_max'] - config['pendulum_mass_min'])

In [ ]:
config

In [ ]:
env_id = 'Hopper-v2'
e = gym.make(env_id)

In [ ]:
e.unwrapped.__dict__.keys()

In [ ]:
e.unwrapped.apply_env_modifications()

In [7]:
a.sim.model.opt

In [41]:
exploit_rewards

[276.1102081018689,
 96.70334286691545,
 85.08844921445562,
 269.074506860484,
 157.6271624984381,
 98.24853913162067,
 285.81629053901,
 47.571903330221204,
 305.7646644475844,
 214.54603825106128,
 318.79671274020046,
 108.78849606101353,
 295.16834090059507,
 304.4609969983021,
 114.21098544869861,
 319.95648068618993,
 381.6618264988649,
 304.1486178093876,
 82.12237601439787,
 266.6843573219772,
 246.96691589862735,
 93.80999550171873,
 317.11151807815327,
 280.0507518589623,
 88.88056611395263,
 56.41848653966207,
 103.50173763966043,
 60.028258282030876,
 110.03073741847325,
 78.99782323464933,
 204.20943212350457,
 63.15781328295692,
 69.76077023209112,
 302.7306806301403,
 372.0257841968678,
 94.7188407595137,
 286.5314683913839,
 292.27559707101716,
 69.19350133789911,
 331.6337563028473,
 49.59573762857002,
 293.9671743156787,
 92.63588041783703,
 338.77067309601244,
 309.0596304951951,
 49.45457787549484,
 115.98180376950395,
 109.69929221601465,
 254.3979814832278,
 310.49